In [75]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

[autoreload of mcr_py failed: Traceback (most recent call last):
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/moritz/miniconda3/envs/mcr-py/lib/python3.10/site-packages/mcr_py/__init__.py", line 3, in <module>
    __doc__ = mcr_py.__doc__
NameError: name 'mcr_py' is not defined
]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
from package.logger import Timed, rlog, setup
from package import storage
setup("INFO")

In [77]:
from mcr_py import run_mlc_with_bags, GraphCache

In [78]:
from copy import deepcopy
from typing import Any, Tuple

import pandas as pd
import geopandas as gpd

import mcr_py
from mcr_py import GraphCache
import pyrosm
from package import storage, strtime
from package.logger import Timed
from package.mcr.label import McRAPTORLabel, merge_intermediate_bags
from package.mcr.path import PathManager, PathType
from package.osm import osm, graph
from package.raptor.mcraptor_single import McRaptorSingle
from package.raptor.bag import Bag
from package.mcr.bag import (
    convert_mc_raptor_bags_to_intermediate_bags,
    convert_mlc_bags_to_intermediate_bags,
)



# ACCURACY = 1
# ACCURACY_MULTIPLIER = 10 ** (ACCURACY - 1)

# AVG_WALKING_SPEED = 1.4  # m/s
# AVG_BIKING_SPEED = 4.0  # m/s


In [79]:
# stops_path = "../data/cleaned/stops.csv"
# city_id = "Koeln"
# osm_path = ""
# structs="../data/structs.pkl"

In [80]:
path_source = "/home/moritz//dev/uni/mcr-py/data/bags.pkl"

data = storage.read_any_dict(path_source)
path_manager = data["path_manager"]
node_map = data["multi_modal_node_to_resetted_map"]
walking_node_map = data["walking_node_to_resetted_map"]
reverse_node_map = {v: k for k, v in node_map.items()}
reverse_walking_node_map = {v: k for k, v in walking_node_map.items()}
stops_df = data["stops_df"]
bags_i = data["bags_i"]


In [81]:
import os
import folium
from package.mcr.path import PathType

In [82]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)

with Timed.info("Reading stops"):
	other_stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, other_stops_df)

[13:38:15] INFO     Reading stops                                                                   ]8;id=832117;file:///tmp/ipykernel_187809/1792129641.py\1792129641.py]8;;\:]8;id=220218;file:///tmp/ipykernel_187809/1792129641.py#5\5]8;;\

           INFO     Reading stops done (0.10 seconds)                                               ]8;id=405589;file:///tmp/ipykernel_187809/1792129641.py\1792129641.py]8;;\:]8;id=200532;file:///tmp/ipykernel_187809/1792129641.py#5\5]8;;\

           INFO     Using existing OSM data                                                        ]8;id=790639;file:///tmp/ipykernel_187809/1792129641.py\1792129641.py]8;;\:]8;id=206794;file:///tmp/ipykernel_187809/1792129641.py#12\12]8;;\

           INFO     Getting OSM graph                                                              ]8;id=17830;file:///tmp/ipykernel_187809/1792129641.py\1792129641.py]8;;\:]8;id=18996;file:///tmp/ipykernel_187809/1792129641.py#16\16]8;;\

           INFO     Loading OSM network from cache                                                 ]8;id=125150;file:///tmp/ipykernel_187809/1792129641.py\1792129641.py]8;;\:]8;id=638334;file:///tmp/ipykernel_187809/1792129641.py#16\16]8;;\

[13:38:16] INFO     Getting OSM graph done (1.49 seconds)                                          ]8;id=868625;file:///tmp/ipykernel_187809/1792129641.py\1792129641.py]8;;\:]8;id=221375;file:///tmp/ipykernel_187809/1792129641.py#16\16]8;;\

In [83]:
labels = pd.DataFrame(
    [
        (label.node_id, label.values[0], label.values[1], n_transfers)
        for n_transfers, bags in bags_i.items()
        for bag in bags.values()
        for label in bag
    ],
    columns=["osm_node_id", "time", "cost", "n_transfers"],
)

labels["human_readable_time"] = labels["time"].apply(strtime.seconds_to_str_time)
labels


,osm_node_id,time,cost,n_transfers,human_readable_time
0,1465634645,31541,0,0,08:45:41
1,250900202,35293,0,0,09:48:13
2,8113090507,33186,0,0,09:13:06
3,1518107548,29131,0,0,08:05:31
4,8512126938,31046,0,0,08:37:26
...,...,...,...,...,...
870545,368892,30500,200,2,08:28:20
870546,3602755781,30312,200,2,08:25:12
870547,3602755781,32547,0,2,09:02:27
870548,3602755781,30887,100,2,08:34:47


In [84]:
labels[["time", "cost", "n_transfers"]].describe()

,time,cost,n_transfers
count,870550.000000,870550.000000,870550.000000
mean,31623.808914,100.452725,1.304738
std,1661.353205,113.290927,0.705664
min,28800.000000,0.000000,0.000000
25%,30345.000000,0.000000,1.000000
50%,31243.000000,100.000000,1.000000
75%,32607.000000,200.000000,2.000000
max,37161.000000,620.000000,2.000000


In [85]:
max_time = strtime.str_time_to_seconds("09:00:00")
max_cost = 10
max_transfers = 10

valid_labels = labels[
	(labels["time"] <= max_time)
	& (labels["cost"] <= max_cost)
	& (labels["n_transfers"] <= max_transfers)
]

In [86]:
selection = valid_labels.sample(min(1000, len(valid_labels)))
# selection = valid_labels

max_cost = selection.cost.max()

from typing import Dict
from colorsys import hls_to_rgb

def hsl_to_hex(h: float, l: float, s: float) -> str:
    r, g, b = [int(x * 255.0) for x in hls_to_rgb(h, l, s)]
    return "#{:02x}{:02x}{:02x}".format(r, g, b)

def generate_colors_hsl(max_cost: float) -> Dict[int, str]:
    if max_cost == 0:
        return {0: "#ffcccc"}
    return {
        cost: hsl_to_hex(h=0, l=0.5 + 0.4 * (max_cost - cost) / max_cost, s=0.8)
        for cost in range(int(max_cost) + 1)
    }

colors = generate_colors_hsl(max_cost)

In [87]:
start_node_id = labels[labels.time == labels.time.min()].iloc[0].osm_node_id


In [88]:
nodes_by_id = nodes.set_index("id")
nodes_by_id["id"] = nodes_by_id.index

start_node = nodes_by_id.loc[start_node_id]

m = folium.Map(location=[start_node.lat, start_node.lon], zoom_start=13)

for label in selection.itertuples():
	node = nodes_by_id.loc[label.osm_node_id]
	folium.CircleMarker(
		location=[node.lat, node.lon],
		radius=3,
		weight=1,
		fill=True,
		fill_color=colors[label.cost],
		fill_opacity=1,
		color="black",
		popup=f"{label.human_readable_time} ({label.cost})",
	).add_to(m)

folium.CircleMarker(
	location=[start_node.lat, start_node.lon],
	radius=5,
	weight=1,
	fill=True,
	fill_color="green",
	fill_opacity=1,
	color="black",
	popup=f"Start",
).add_to(m)

m

In [89]:
translator_map = {
    PathType.WALKING: reverse_walking_node_map,
    PathType.CYCLING_WALKING: reverse_node_map,
    PathType.PUBLIC_TRANSPORT: None,
}
no_prefix_reverse_walking_node_map = {
    k: int(v[1:]) for k, v in reverse_walking_node_map.items()
}
no_prefix_reverse_node_map = {k: int(v[1:]) for k, v in reverse_node_map.items()}
no_prefix_translator_map = {
    PathType.WALKING: no_prefix_reverse_walking_node_map,
    PathType.CYCLING_WALKING: no_prefix_reverse_node_map,
    PathType.PUBLIC_TRANSPORT: None,
}


In [90]:
from package.mcr.path import Path, GTFSPath

In [91]:
stops_df['stop_id'] = stops_df['stop_id'].astype(int)
stops_by_id = stops_df.set_index('stop_id')
stops_by_id.head(2)

,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,geometry,nearest_node,nearest_node_dist
stop_id,,,,,,,,,,,,,
1,,Köln Heumarkt,,50.935705,6.959995,,,0,,,POINT (6.96000 50.93570),673702632,16.337340
2,,Köln Neumarkt,,50.93577,6.947677,,,0,,,POINT (6.94768 50.93577),443861,17.695676


In [92]:
walking_result_bags = bags_i[2]

In [93]:
walking_result_bags_flat = []
for node_id, bag in walking_result_bags.items():
	for label in bag:
		walking_result_bags_flat.append((node_id, label))

In [94]:
path_objs_with_ids = pd.Series(
    list(
        map(
            lambda x: (
                x[0],
                path_manager.reconstruct_and_translate_path_for_label(
                    x[1], no_prefix_translator_map
                ),
            ),
            walking_result_bags_flat[:10000],
        )
    )
)
node_ids, path_objs = list(zip(*path_objs_with_ids))
path_objs[:3]

([Path(path_type=PathType.WALKING, path=[394001227, 394001227, 256204817, 8805331133, 3251172951, 3251172939, 256204816, 3251172916, 3251172913, 256204815, 3251172921, 266714877, 3251172936, 734098317, 3251173087, 256204814, 3555564977, 266709899, 622130519, 4297860762, 266710176, 4297860761, 256204683, 737255999, 4297860760, 8541426514, 11092001174, 8541426513, 256204684, 8541426512, 11092001175, 2321027922, 2321027920, 256544163, 633363546, 256544162, 2321027913, 633363544, 2321028674, 1828831138, 3508688716, 3688544166, 259079185, 3536219151, 246466022, 246465990, 1829256584, 1829256586, 1829256581, 246465989, 1829256583, 1829256570, 9359082673, 1829256567, 1829256563, 1829256558, 1829256557, 3662622074, 3542256445, 1829256590, 1829256588, 3662622075, 3523003910, 9359082666, 3523003916, 3660228862, 3660215682, 2302638239, 2302638246, 2302638251, 647131965, 1679798099, 1679798061, 1679798045, 1679798041, 1679798017, 640393040, 1679798012, 2302465580, 1239333800, 6087584263, 640393030

In [95]:
len(walking_result_bags_flat)

389817

In [96]:
node_ids = pd.Series(node_ids)
path_objs = pd.Series(list(path_objs))

In [97]:
df = pd.DataFrame(path_objs, columns=["paths"])
df["contains_gtfs"] = df["paths"].apply(
    lambda x: any([isinstance(path, GTFSPath) for path in x])
)
df["first_path_length"] = df["paths"].apply(lambda x: len(x[0].path))
df["third_path_length"] = df["paths"].apply(lambda x: len(x[2].path))
df["fourth_path_length"] = df["paths"].apply(lambda x: len(x[3].path) if isinstance(x[3], Path) else 0)
df["fifth_path_length"] = df["paths"].apply(lambda x: len(x[4].path))
# df["sixth_path_length"] = df["paths"].apply(lambda x: len(x[5].path) if isinstance(x[5], Path) else 0)
# df["seventh_path_length"] = df["paths"].apply(lambda x: len(x[6].path))
df["path_types"] = df["paths"].apply(
    lambda x: [path.path_type if isinstance(path, Path) else "GTFS" for path in x]
)
df["node_id"] = node_ids
df.head(3)


,paths,contains_gtfs,first_path_length,third_path_length,fourth_path_length,fifth_path_length,path_types,node_id
0,"[Path(path_type=PathType.WALKING, path=[394001...",False,82,0,182,0,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",1465561514
1,"[Path(path_type=PathType.WALKING, path=[394001...",False,13,0,263,0,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",1465561514
2,"[Path(path_type=PathType.WALKING, path=[394001...",False,13,0,263,0,"[PathType.WALKING, PathType.CYCLING_WALKING, P...",1465561514


In [98]:
df[df.contains_gtfs].size / df.size

0.1939

In [99]:
df[df.contains_gtfs].node_id.unique().size / df.node_id.unique().size

0.39540012602394453

In [100]:
df[df.contains_gtfs]

,paths,contains_gtfs,first_path_length,third_path_length,fourth_path_length,fifth_path_length,path_types,node_id
6,"[Path(path_type=PathType.WALKING, path=[394001...",True,16,23,312,0,"[PathType.WALKING, GTFS, PathType.WALKING, Pat...",310636273
17,"[Path(path_type=PathType.WALKING, path=[394001...",True,16,29,74,0,"[PathType.WALKING, GTFS, PathType.WALKING, Pat...",357124
18,"[Path(path_type=PathType.WALKING, path=[394001...",True,16,0,0,6,"[PathType.WALKING, GTFS, PathType.WALKING, GTF...",357124
27,"[Path(path_type=PathType.WALKING, path=[394001...",True,16,0,0,145,"[PathType.WALKING, GTFS, PathType.WALKING, GTF...",7978775102
29,"[Path(path_type=PathType.WALKING, path=[394001...",True,16,0,0,85,"[PathType.WALKING, GTFS, PathType.WALKING, GTF...",2492628510
...,...,...,...,...,...,...,...,...
9982,"[Path(path_type=PathType.WALKING, path=[394001...",True,16,23,329,0,"[PathType.WALKING, GTFS, PathType.WALKING, Pat...",999471696
9983,"[Path(path_type=PathType.WALKING, path=[394001...",True,16,0,0,118,"[PathType.WALKING, GTFS, PathType.WALKING, GTF...",999471696
9987,"[Path(path_type=PathType.WALKING, path=[394001...",True,16,0,0,74,"[PathType.WALKING, GTFS, PathType.WALKING, GTF...",999430666
9988,"[Path(path_type=PathType.WALKING, path=[394001...",True,16,8,285,0,"[PathType.WALKING, GTFS, PathType.WALKING, Pat...",999430666


In [101]:
effective_by_public_transport = df[df.contains_gtfs].node_id.unique()

In [102]:
import random
i = random.sample(list(df.node_id.unique()), 1)[0]
# i = 113757
sel = [tu for tu in walking_result_bags_flat if tu[0] == i]
print(f"num GTFS paths = {df[df.node_id == i].contains_gtfs.sum()}")
print(i)

num GTFS paths = 0
1686308767


In [103]:
print("\n".join(map(str, walking_result_bags[i])))

IntermediateLabel(values=[33693, 0], hidden_values=[0], path=[23890, 468046, 710194, 1152433, 1249038], node_id=1686308767)
IntermediateLabel(values=[30590, 200], hidden_values=[0], path=[23890, 468046, 710194, 1152434, 1249039], node_id=1686308767)
IntermediateLabel(values=[31832, 100], hidden_values=[0], path=[23890, 468046, 710194, 1152435, 1249040], node_id=1686308767)


In [104]:
from folium import plugins
from folium.plugins import HeatMap

In [105]:
start_time = strtime.str_time_to_seconds("08:00:00")

In [106]:
def format_meta(meta, previous_meta):
	values = meta["values"]
	arrival_time = values[0]
	cost = values[1]

	if previous_meta:
		previous_values = previous_meta["values"]
		previous_arrival_time = previous_values[0]
		previous_cost = previous_values[1]
		
		arrival_time -= previous_arrival_time
		cost -= previous_cost
	else:
		arrival_time -= start_time

	return f"{strtime.seconds_to_str_time(arrival_time)} ({cost})"

In [112]:
toloop = sel

sample_label = sel[0][1]
sample_node_id = sample_label.node_id
nodes_by_id = nodes.set_index("id")
nodes_by_id["id"] = nodes_by_id.index
sample_node = nodes_by_id.loc[sample_node_id]

m = folium.Map(location=[sample_node.lat, sample_node.lon], zoom_start=13)

for end_node_id , label in toloop:

    end_node = nodes_by_id.loc[end_node_id]

    folium.CircleMarker(
        location=[end_node.lat, end_node.lon],
        popup=f"End: {end_node_id}",
        color="red",
        radius=3,
    ).add_to(m)

    paths = path_manager.reconstruct_and_translate_path_for_label(
        label, translator_map
    )
    for i, path in enumerate(paths):
        if isinstance(path, Path):
            if path.path == []:
                continue
            cycling_path_nodes = [nodes_by_id.loc[int(node_id[1:])] for node_id in path.path if node_id[0] == "B"]
            walking_path_nodes = [nodes_by_id.loc[int(node_id[1:])] for node_id in path.path if node_id[0] == "W"]
            # print(f"cycling length: {len(cycling_path_nodes)}")
            # print(f"walking length: {len(walking_path_nodes)}\n")
            path_lat_lon = [(node.lat, node.lon) for node in cycling_path_nodes]
            previous_meta = paths[i-1].meta if i > 0 else None
            meta = format_meta(path.meta, previous_meta)
            if path_lat_lon != []:
                folium.PolyLine(path_lat_lon, color="blue", weight=2, popup=str(meta)).add_to(m)
            path_lat_lon = [(node.lat, node.lon) for node in walking_path_nodes]
            if path_lat_lon != []:
                folium.PolyLine(path_lat_lon, color="red", weight=2, popup=str(meta)).add_to(m)
        elif isinstance(path, GTFSPath):
            start_stop_id = path.start_stop_id
            end_stop_id = path.end_stop_id
            start_stop = stops_by_id.loc[start_stop_id]
            end_stop = stops_by_id.loc[end_stop_id]
            trip = path.trip_id

            path_lat_lon = [
                (float(start_stop.stop_lat), float(start_stop.stop_lon)),
                (float(end_stop.stop_lat), float(end_stop.stop_lon)),
            ]
            folium.PolyLine(
                path_lat_lon,
                color="green",
                weight=2,
                popup=f"Trip: {trip}",
            ).add_to(m)

            folium.CircleMarker(
                location=[float(start_stop.stop_lat), float(start_stop.stop_lon)],
                popup=f"Start: {start_stop.stop_name}",
                color="green",
                radius=3,
            ).add_to(m)
            folium.CircleMarker(
                location=[float(end_stop.stop_lat), float(end_stop.stop_lon)],
                popup=f"End: {end_stop.stop_name}",
                color="green",
                radius=3,
            ).add_to(m)
        else:
            raise Exception("Unknown path type")

m


In [115]:
m = folium.Map(location=[sample_node.lat, sample_node.lon], zoom_start=13)

heat_map_nodes = [nodes_by_id.loc[node_id] for node_id in effective_by_public_transport]
HeatMap(
    [(node.lat, node.lon) for node in heat_map_nodes],
).add_to(m)

for _, stop in stops_df.iterrows():
    node = nodes_by_id.loc[stop["nearest_node"]]

    folium.CircleMarker(
        location=[node.lat, node.lon],
        radius=3,
        popup=f"node_id: {node_id}",
        color="green",
    ).add_to(m)


m

: 